# Concise Implementation of Linear Regression

We will show you how to implement
the linear regression model
concisely by using high-level APIs of deep learning frameworks.
To start, we will generate the same dataset

In [2]:
import numpy as np
import torch
from torch.utils import data

from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

Rather than rolling our own iterator,
we can call upon the existing API in a framework to read data

In [5]:
def load_array(data_arrays, batch_size, is_train=True):  
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.0438,  1.3844],
         [ 2.0319, -0.1047],
         [-0.9132, -1.7336],
         [-0.1785,  0.1843],
         [-0.2485, -1.1044],
         [ 0.6430,  0.1515],
         [-1.1161,  1.5691],
         [ 0.5905,  0.8691],
         [ 0.0371, -0.2041],
         [-1.1344, -1.8581]]),
 tensor([[-0.4198],
         [ 8.6138],
         [ 8.2665],
         [ 3.2123],
         [ 7.4729],
         [ 4.9707],
         [-3.3838],
         [ 2.4409],
         [ 4.9576],
         [ 8.2429]])]

We can use a framework's predefined layers,
which allow us to focus especially
on the layers used to construct the model
rather than having to focus on the implementation

In [6]:
# `nn` is an abbreviation for neural networks
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

Before using `net`, we need to initialize the model parameters,
such as the weights and bias in the linear regression model

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

The `MSELoss` class computes the mean squared error, also known as squared $L_2$ norm.
By default it returns the average loss over examples

In [8]:
loss = nn.MSELoss()

When we instantiate an `SGD` instance,
we will specify the parameters to optimize over

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

Once we have all the basic pieces in place,
the training loop itself is strikingly similar
to what we did when implementing everything from scratch

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000191
epoch 2, loss 0.000099
epoch 3, loss 0.000099


We compare the model parameters learned by training on finite data
and the actual parameters that generated our dataset

In [11]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([ 0.0005, -0.0005])
error in estimating b: tensor([-5.6744e-05])
